<a href="https://colab.research.google.com/github/iamhb/BITS_Final_SEM/blob/main/Final_Project_Execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install & Imports

In [1]:
!pip install -q psycopg2-binary sqlalchemy
!pip install -q langchain langchain-community chromadb
!pip install -q sentence-transformers
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import psycopg2

Device

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

DB connection

In [4]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

DB_USER = "postgres.hxwwrvlxbvrmhbvdvmbd"
DB_PASSWORD = "B!T$@F!n4L_Pr0J3cT#"
DB_HOST = "aws-1-ap-southeast-2.pooler.supabase.com"
DB_PORT = "6543"
DB_NAME = "postgres"

DB_PASSWORD_ENCODED = quote_plus(DB_PASSWORD)

DATABASE_URL = (
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD_ENCODED}"
    f"@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

engine = create_engine(
    DATABASE_URL,
    pool_pre_ping=True,
    pool_size=5,
    max_overflow=5,
)


Testing DB connection

In [6]:
with engine.connect() as conn:
    print(conn.execute(text("SELECT 1")).all())

[(1,)]


Execute SQL

In [11]:
def execute_sql(sql):
    try:
        with engine.connect() as conn:
            result = conn.execute(text(sql))
            rows = result.fetchall()
            return rows, True
    except Exception as e:
        print(f"SQL Error: {e}")
        return str(e), False

Model name

In [12]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

Test Questionss

In [13]:
questions = [
    "Show all customers",
    "Count number of orders",
    "List orders placed in 2024",
    "Show total sales per customer",
    "Find customers with no orders"
]

======================================================Experiment A======================================================
==========================================Baseline model - Non Quantized, No Schema ====================================

Load Baseline Model

In [14]:
tokenizer_base = AutoTokenizer.from_pretrained(MODEL_NAME)
model_base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Bseline Prompt Builder

In [15]:
def build_baseline_prompt(question):
    return f"""
        Translate the following question into SQL.

        Question: {question}

        Return only the SQL query.
"""

Run Baseline Experiment

In [16]:
baseline_results = []

for q in questions:
    prompt_base = build_baseline_prompt(q)

    inputs_base = tokenizer_base(
        prompt_base,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs_base = model_base.generate(
            **inputs_base,
            max_new_tokens=256,
            do_sample=False
        )

    sequences_base = outputs_base
    gen_tokens_base = sequences_base[:, inputs_base["input_ids"].shape[-1]:]

    sql_base = tokenizer_base.decode(
        gen_tokens_base[0],
        skip_special_tokens=True
    ).strip()

    result, success = execute_sql(sql_base)

    baseline_results.append({
        "question": q,
        "sql": sql_base,
        "success": success
    })

print("baseline_results")
print(baseline_results)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SQL Error: (psycopg2.errors.SyntaxError) syntax error at or near "Note"
LINE 3:         Note: The table name is "customers" and it has colum...
                ^

[SQL: SELECT * FROM customers;

        Note: The table name is "customers" and it has columns: id, name, email, phone, address, city, state, zip, country.

        SELECT id, name, email, phone, address, city, state, zip, country FROM customers;]
(Background on this error at: https://sqlalche.me/e/20/f405)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SQL Error: (psycopg2.errors.SyntaxError) syntax error at or near "This"
LINE 3:         This query will count the number of rows in the 'ord...
                ^

[SQL: SELECT COUNT(*) FROM orders;

        This query will count the number of rows in the 'orders' table.]
(Background on this error at: https://sqlalche.me/e/20/f405)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SQL Error: (psycopg2.errors.SyntaxError) syntax error at or near "Note"
LINE 3:         Note: Assuming that the orders table has a column na...
                ^

[SQL: SELECT * FROM orders WHERE year(order_date) = 2024;

        Note: Assuming that the orders table has a column named order_date.

        This query will return all rows from the orders table where the year of the order_date is 2024. The year() function is used to extract the year from a date value.]
(Background on this error at: https://sqlalche.me/e/20/f405)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


SQL Error: (psycopg2.errors.UndefinedTable) relation "sales" does not exist
LINE 2:         FROM sales
                     ^

[SQL: SELECT customer_id, SUM(sales) AS total_sales
        FROM sales
        GROUP BY customer_id;]
(Background on this error at: https://sqlalche.me/e/20/f405)
baseline_results
[{'question': 'Show all customers', 'sql': 'SELECT * FROM customers;\n\n        Note: The table name is "customers" and it has columns: id, name, email, phone, address, city, state, zip, country.\n\n        SELECT id, name, email, phone, address, city, state, zip, country FROM customers;', 'success': False}, {'question': 'Count number of orders', 'sql': "SELECT COUNT(*) FROM orders;\n\n        This query will count the number of rows in the 'orders' table.", 'success': False}, {'question': 'List orders placed in 2024', 'sql': 'SELECT * FROM orders WHERE year(order_date) = 2024;\n\n        Note: Assuming that the orders table has a column named order_date.\n\n        This query will re

Cleanup Baseline Model

In [ ]:
del model_base
torch.cuda.empty_cache()